In [1]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

In [3]:
dataset, info = tfds.load('ag_news_subset', with_info=True, as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/ag_news_subset/1.0.0.incompleteC46YI2/ag_news_subset-train.tfrecord


  0%|          | 0/120000 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/ag_news_subset/1.0.0.incompleteC46YI2/ag_news_subset-test.tfrecord


  0%|          | 0/7600 [00:00<?, ? examples/s]

Dataset ag_news_subset downloaded and prepared to /root/tensorflow_datasets/ag_news_subset/1.0.0. Subsequent calls will reuse this data.


In [4]:
dataset

{'test': <PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>,
 'train': <PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>}

In [5]:
train_data = dataset['train']
test_data = dataset['test']

In [7]:
for sentence, label in train_data:
  print(sentence)
  print(label)
  break

tf.Tensor(b'AMD #39;s new dual-core Opteron chip is designed mainly for corporate computing applications, including databases, Web services, and financial transactions.', shape=(), dtype=string)
tf.Tensor(3, shape=(), dtype=int64)


In [12]:
train_sentences = []
train_labels = []

test_sentences = []
test_labels = []

for sentence, label in train_data:
  train_sentences.append(sentence.numpy().decode('utf8'))
  train_labels.append(label.numpy())

for sentence, label in test_data:
  test_sentences.append(sentence.numpy().decode('utf8'))
  test_labels.append(label.numpy())

In [14]:
vocab_size = 10000
embedding_dim = 16
max_length = 100
trunc_type = 'post'
pad_type = 'post'
oov_token = '<oov>'

In [15]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(train_sentences)

word_index = tokenizer.word_index

sequences = tokenizer.texts_to_sequences(train_sentences)
padded_sequences = pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)

test_sequences = tokenizer.texts_to_sequences(test_sentences)
padded_test_sequences = pad_sequences(test_sequences, maxlen=max_length, truncating=trunc_type)

train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [29]:
model = tf.keras.models.Sequential([
                                    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
                                    #tf.keras.layers.LSTM(32, return_sequences=True),
                                    tf.keras.layers.LSTM(16),
                                    tf.keras.layers.Dropout(0.2),
                                    tf.keras.layers.Dense(8, activation='relu'),
                                    tf.keras.layers.Dense(4, activation='softmax')
])

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 100, 16)           160000    
                                                                 
 lstm_5 (LSTM)               (None, 16)                2112      
                                                                 
 dropout_3 (Dropout)         (None, 16)                0         
                                                                 
 dense_6 (Dense)             (None, 8)                 136       
                                                                 
 dense_7 (Dense)             (None, 4)                 36        
                                                                 
Total params: 162,284
Trainable params: 162,284
Non-trainable params: 0
_________________________________________________________________


In [30]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [31]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_accuracy', patience=3, min_delta=0.01)

In [32]:
history = model.fit(padded_sequences,
                    train_labels,
                    validation_data=(padded_test_sequences, test_labels),
                    epochs=30,
                    callbacks=early_stopping)

Epoch 1/30
3750/3750 [==============================] - 88s 23ms/step - loss: 0.5091 - accuracy: 0.8091 - val_loss: 0.3265 - val_accuracy: 0.8954
Epoch 2/30
3750/3750 [==============================] - 85s 23ms/step - loss: 0.2848 - accuracy: 0.9114 - val_loss: 0.3030 - val_accuracy: 0.9022
Epoch 3/30
3750/3750 [==============================] - 85s 23ms/step - loss: 0.2427 - accuracy: 0.9229 - val_loss: 0.3002 - val_accuracy: 0.9024
Epoch 4/30
3750/3750 [==============================] - 85s 23ms/step - loss: 0.2143 - accuracy: 0.9317 - val_loss: 0.3065 - val_accuracy: 0.9025


In [33]:
model.save('model.h5')